In [70]:
import pandas as pd
from pandas import DataFrame
import dateutil.parser
import json


view_party=pd.read_csv('../refdata/view_party.csv').drop(['cmp','euprofiler','ees','castles_mair','huber_inglehart','ray','benoit_laver','chess'],axis=1)
partyColumns=view_party.columns

view_election=pd.read_csv('../refdata/view_election.csv')
electionColumns=view_election.columns

view_election=view_election.merge(view_party,on='party_id',how='left',suffixes=('','_2'))
view_election=view_election.drop([x for x in view_election.columns if x.endswith('_2')],axis=1)
view_election.election_date=view_election.election_date.apply(dateutil.parser.parse)

view_cabinet=pd.read_csv('../refdata/view_cabinet.csv')
cabinetColumns=view_cabinet.columns

view_cabinet=view_cabinet.merge(view_party,on='party_id',how='left',suffixes=('','_2'))
view_cabinet=view_cabinet.drop([x for x in view_cabinet.columns if x.endswith('_2')],axis=1)
view_cabinet=view_cabinet.merge(view_election,on=['election_id','party_id'],how='left',suffixes=('','_2'))
view_cabinet=view_cabinet.drop([x for x in view_cabinet.columns if x.endswith('_2')],axis=1)
view_cabinet.election_date=view_cabinet.election_date.apply(dateutil.parser.parse)


NationalElections=view_election[view_election.election_type=='parliament']
EuroElections=view_election[view_election.election_type=='ep']

def getLastElection(t):
    return t[t.election_date==t.election_date.max()]

def getLastCabinet(t):
    return t[t.start_date==t.start_date.max()]

NationalElections=NationalElections.groupby('country_name_short').apply(getLastElection).reset_index(drop=True)
EuroElections=EuroElections.groupby('country_name_short').apply(getLastElection).reset_index(drop=True)
Cabinets=view_cabinet.groupby('country_name_short').apply(getLastCabinet).reset_index(drop=True)

NationalElections=NationalElections[~NationalElections.country_name_short.isin(['AUS','CAN','ISR','NZL','JPN'])]
EuroElections=EuroElections[~EuroElections.country_name_short.isin(['AUS','CAN','ISR','NZL','JPN'])]
Cabinets=Cabinets[~Cabinets.country_name_short.isin(['AUS','CAN','ISR','NZL','JPN'])].drop(['election_type','previous_cabinet_id','previous_parliament_election_id'],axis=1)

NationalElections.to_csv('../refdata/national_elections.csv',index=False)
EuroElections.to_csv('../refdata/euro_elections.csv',index=False)
Cabinets.to_csv('../refdata/cabinets.csv',index=False)

def getPartyJson(code):
    topLevelFields=['country_id','country_name_short','country_name','election_id','election_date','seats_total',
                   'cabinet_id','cabinet_name','start_date','caretaker','election_seats_total']
    
    nat = Cabinets[code==Cabinets.country_name_short] \
        .sort_values(['seats','vote_share','party_name_short'],ascending=False)
    nat.seats_total=nat.seats_total.fillna(-1)
    r=nat[topLevelFields].reset_index(drop=True).ix[0].to_dict()
    r['election_date']=str(r['election_date'])
    r.update({'parties' : [x[1][nat.columns.difference(topLevelFields)].dropna().to_dict() for x in nat.iterrows()]})
    
    if code in CPI_2015_data.wbcode.tolist():
        r['cpi2015'] = CPI_2015_data[CPI_2015_data.wbcode==code].CPI2015.tolist()[0]
    if code in WDI_data.countryCode.tolist():
        r.update(WDI_data[WDI_data.countryCode==code].reset_index(drop=True).T.drop('countryCode')[0].dropna().to_dict())
    return r

data={code:getPartyJson(code) for code in Cabinets['country_name_short'].drop_duplicates() if not code in ['AUS','CAN','ISR','NZL','JPN']}

with open('../public/parlgov.json', 'w') as outfile:
    json.dump(data, outfile,indent=4, default='')

In [24]:
Cabinets[Cabinets.country_name=='Germany']

Cabinets[Cabinets.caretaker==1]


,country_name_short,country_name,election_date,start_date,cabinet_name,caretaker,cabinet_party,prime_minister,seats,election_seats_total,...,family_name_short,family_name,state_market,liberty_authority,eu_anti_pro,family_id,election_type,vote_share,seats_total,previous_parliament_election_id
85,ESP,Spain,2015-12-20,2015-12-20,Rajoy II,1,1,1,123.0,350,...,con,Conservative,7.6190,7.8127,7.7368,26.0,parliament,28.72,350.0,777.0
86,ESP,Spain,2015-12-20,2015-12-20,Rajoy II,1,0,0,40.0,350,...,soc,Social democracy,3.5000,3.5000,8.1000,11.0,parliament,13.93,350.0,777.0
87,ESP,Spain,2015-12-20,2015-12-20,Rajoy II,1,0,0,1.0,350,...,spec,Special issue,5.9000,5.7694,8.0278,16.0,parliament,0.33,350.0,777.0
88,ESP,Spain,2015-12-20,2015-12-20,Rajoy II,1,0,0,8.0,350,...,con,Conservative,6.4000,6.9000,7.8000,26.0,parliament,2.25,350.0,777.0
89,ESP,Spain,2015-12-20,2015-12-20,Rajoy II,1,0,0,12.0,350,...,com,Communist/Socialist,1.4000,3.0000,3.3000,14.0,parliament,3.69,350.0,777.0
90,ESP,Spain,2015-12-20,2015-12-20,Rajoy II,1,0,0,2.0,350,...,spec,Special issue,NaN,NaN,NaN,16.0,parliament,0.87,350.0,777.0
91,ESP,Spain,2015-12-20,2015-12-20,Rajoy II,1,0,0,6.0,350,...,com,Communist/Socialist,1.4000,3.0000,3.3000,14.0,parliament,1.63,350.0,777.0
92,ESP,Spain,2015-12-20,2015-12-20,Rajoy II,1,0,0,9.0,350,...,spec,Special issue,2.9073,2.7325,7.0389,16.0,parliament,2.39,350.0,777.0
93,ESP,Spain,2015-12-20,2015-12-20,Rajoy II,1,0,0,9.0,350,...,com,Communist/Socialist,1.4000,3.0000,3.3000,14.0,parliament,2.67,350.0,777.0
94,ESP,Spain,2015-12-20,2015-12-20,Rajoy II,1,0,0,2.0,350,...,com,Communist/Socialist,1.8937,1.7351,5.2186,14.0,parliament,3.67,350.0,777.0


In [29]:
CPI_2015_data=pd.read_excel('../build/CPI_2015_data.xlsx')
CPI_2015_data=CPI_2015_data[CPI_2015_data.Region.isin(['ECA','WE/EU'])]

WDI_data=pd.read_csv('../build/WDI_data.csv')
WDI_data=WDI_data[['Country Code','Indicator Code','2015']][WDI_data['Country Code'].isin(CPI_2015_data.wbcode)] \
    .set_index(['Country Code','Indicator Code']).unstack()['2015'][['SP.POP.TOTL','NY.GDP.MKTP.CD', 'NY.GDP.PCAP.CD']] \
    .reset_index().rename(columns={
            'Country Code':'countryCode',
            'SP.POP.TOTL':'totalPopulation',
            'NY.GDP.MKTP.CD':'gdpUsd',
            'NY.GDP.PCAP.CD':'gdpPerCapitaUsd',
        }).rename_axis(None,axis=1)

In [68]:
code='AUT'

if code in CPI_2015_data.wbcode:
    print 1

WDI_data[WDI_data.countryCode==code].reset_index(drop=True).T.drop('countryCode')[0].to_dict()

{'gdpPerCapitaUsd': 43438.8630381343,
 'gdpUsd': 374055872241.322,
 'totalPopulation': 8611088.0}

In [75]:
WDI_data['gdpUsdCalc']=WDI_data['gdpPerCapitaUsd']*WDI_data['totalPopulation']
WDI_data['gdpUsdCalc']/WDI_data.gdpUsdCalc

0     1.0
1     1.0
2     1.0
3     1.0
4     1.0
5     1.0
6     1.0
7     1.0
8     1.0
9     1.0
10    1.0
11    1.0
12    1.0
13    1.0
14    1.0
15    1.0
16    1.0
17    1.0
18    1.0
19    1.0
20    1.0
21    1.0
22    1.0
23    1.0
24    1.0
25    1.0
26    1.0
27    1.0
28    1.0
29    1.0
30    1.0
31    1.0
32    NaN
33    1.0
34    1.0
35    1.0
36    1.0
37    1.0
38    1.0
39    1.0
40    1.0
41    1.0
42    1.0
43    1.0
44    1.0
45    1.0
46    1.0
dtype: float64